In [53]:
import pandas as pd

csv_list = [
    'OCR Dataset - Bank Jago.csv',
    'OCR Dataset - Shopee Pay.csv',
    'OCR Dataset - BLU.csv',
    'OCR Dataset - Gopay.csv',
    'OCR Dataset - Livin By Mandiri.csv',
    'OCR Dataset - MyBCA.csv',
    'OCR Dataset - MyBCA SS.csv',
    'OCR Dataset - OVO.csv',
    'OCR Dataset - Donasi sintesis.csv',
    'OCR Dataset - Bills Sintesis.csv',
    'OCR Dataset - Health Sintesis.csv',
    'OCR Dataset - Education Sintesis.csv',
    'OCR Dataset - Entertainment Sintesis.csv',
    'OCR Dataset - Bills Sintesis 2.csv',
    'OCR Dataset - Donations Sintesis 2.csv',
    'OCR Dataset - Education Sintesis 2.csv',
    'OCR Dataset - Entertainment Sintesis 2.csv',
    'OCR Dataset - FnB Sintesis 2.csv',
    'OCR Dataset - Health Sintesis 2.csv',
    'OCR Dataset - Shopping Sintesis 2.csv',
    'OCR Dataset - Transport Sintesis 2.csv',
    'OCR Dataset - Savings and Investment.csv'
]

dfs = []

for csv_file in csv_list:
    df = pd.read_csv('datasets/' + csv_file)
    df['source'] = csv_file 
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)

df


,Nama File,Hasil OCR,Kategori,Method,source
0,Jago_01,Dimsum Nat food\nRp13.500\nID Transaksi\n23878...,Food and Beverages,QRIS,OCR Dataset - Bank Jago.csv
1,Jago_02,KIOS TALENTA\nRp2.200\nID Transaksi\n234014770...,Shopping,QRIS,OCR Dataset - Bank Jago.csv
2,Jago_03,Jus Mbak Yuli\nRp11.000\nID Transaksi\n2365882...,Food and Beverages,QRIS,OCR Dataset - Bank Jago.csv
3,Jago_04,KEDAI YO\nRp6.000\nID Transaksi\n2350877O19\nS...,Food and Beverages,QRIS,OCR Dataset - Bank Jago.csv
4,Jago_05,"SUMARNI, Otomotif\nBOGOR\nRp13.000\nID Transak...",Transport,QRIS,OCR Dataset - Bank Jago.csv
...,...,...,...,...,...
1264,blu_220,Investasi Pendapatan Tetap\n Transaksi Berhasi...,Savings and Investment,Qris,OCR Dataset - Savings and Investment.csv
1265,blu_221,Pembelian Unit Penyertaan\n Transaksi Berhasil...,Savings and Investment,Transfer,OCR Dataset - Savings and Investment.csv
1266,blu_222,Top Up Tabungan\n Transaksi Berhasil\n 16 Sep ...,Savings and Investment,Transfer,OCR Dataset - Savings and Investment.csv
1267,blu_223,Setoran Deposito\n Transaksi Berhasil\n 05 Nov...,Savings and Investment,Transfer,OCR Dataset - Savings and Investment.csv


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1269 entries, 0 to 1268
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Nama File  1269 non-null   object
 1   Hasil OCR  1269 non-null   object
 2   Kategori   1268 non-null   object
 3   Method     1268 non-null   object
 4   source     1269 non-null   object
dtypes: object(5)
memory usage: 49.7+ KB


In [55]:
# cek yang null siapa
df.isnull().sum()

Nama File    0
Hasil OCR    0
Kategori     1
Method       1
source       0
dtype: int64

In [56]:
df[df.isnull().any(axis=1)]

,Nama File,Hasil OCR,Kategori,Method,source
235,ERROR,ERROR,NaN,NaN,OCR Dataset - Livin By Mandiri.csv


In [57]:
df[df['Nama File'] == 'Nama File']

,Nama File,Hasil OCR,Kategori,Method,source


In [58]:
#drop yang null
df = df.dropna().reset_index(drop=True)

df.isnull().sum()

Nama File    0
Hasil OCR    0
Kategori     0
Method       0
source       0
dtype: int64

In [59]:
df['Kategori'].value_counts()

Kategori
Food and Beverages        150
Shopping                  148
Transport                 131
Bills                     131
Donations                 127
Education                 125
Savings and Investment    125
Health                    123
Other                      79
Entertaiment               79
Entertainment              50
Name: count, dtype: int64

In [60]:
df['Method'].value_counts()

Method
E-Wallet           294
Transfer           201
QRIS               190
Qris               190
E-WALLET           180
TRANSFER           138
Virtual Account     50
VA                  25
Name: count, dtype: int64

In [61]:
# karena Bills, Entertaiment, Education, Health sedikit, kita gabung menjadi Lifestyle/Reccurring
# df['Kategori'] = df['Kategori'].replace({
#     'Bills': 'Lifestyle/Reccurring',
#     'Entertaiment': 'Lifestyle/Reccurring',
# })

# typo entertaiment ke entertainment
df['Kategori'] = df['Kategori'].replace({
    'Entertaiment': 'Entertainment',
})

In [62]:
#delete all rows with Lifestyle/Reccurring
df = df[df['Kategori'] != 'Lifestyle/Reccurring'].reset_index(drop=True)

In [63]:
df['Kategori'].value_counts()

Kategori
Food and Beverages        150
Shopping                  148
Transport                 131
Bills                     131
Entertainment             129
Donations                 127
Education                 125
Savings and Investment    125
Health                    123
Other                      79
Name: count, dtype: int64

In [64]:
import re

def clean_ocr_basic(text):
    text = str(text)
    text = text.lower()
    text = text.replace("\n", " ")
    text = re.sub(r"\s+", " ", text)   # spasi ganda
    return text.strip()

def remove_ui_noise(text):
    noise_patterns = [
        r"id transaksi",
        r"transaksi berhasil",
        r"success",
        r"ref no",
    ]
    for p in noise_patterns:
        text = re.sub(p, " ", text)
    return re.sub(r"\s+", " ", text).strip()


def clean_ocr(text):
    text = clean_ocr_basic(text)
    text = remove_ui_noise(text)
    return text

df['Hasil OCR'] = df['Hasil OCR'].apply(clean_ocr)
df


,Nama File,Hasil OCR,Kategori,Method,source
0,Jago_01,dimsum nat food rp13.500 2387821914 sumber aku...,Food and Beverages,QRIS,OCR Dataset - Bank Jago.csv
1,Jago_02,kios talenta rp2.200 2340147702 sumber akun ke...,Shopping,QRIS,OCR Dataset - Bank Jago.csv
2,Jago_03,jus mbak yuli rp11.000 2365882380 sumber akun ...,Food and Beverages,QRIS,OCR Dataset - Bank Jago.csv
3,Jago_04,kedai yo rp6.000 2350877o19 sumber akun kezia ...,Food and Beverages,QRIS,OCR Dataset - Bank Jago.csv
4,Jago_05,"sumarni, otomotif bogor rp13.000 2574180843 su...",Transport,QRIS,OCR Dataset - Bank Jago.csv
...,...,...,...,...,...
1263,blu_220,investasi pendapatan tetap 26 okt 2025 | 08:12...,Savings and Investment,Qris,OCR Dataset - Savings and Investment.csv
1264,blu_221,pembelian unit penyertaan pembelian instrumen ...,Savings and Investment,Transfer,OCR Dataset - Savings and Investment.csv
1265,blu_222,top up tabungan 16 sep 2025 | 11:35:01 wib tot...,Savings and Investment,Transfer,OCR Dataset - Savings and Investment.csv
1266,blu_223,setoran deposito 05 nov 2025| 03:35:32 wib nom...,Savings and Investment,Transfer,OCR Dataset - Savings and Investment.csv


In [65]:
from sklearn.model_selection import train_test_split

X = df[["Hasil OCR"]]
# X = df ["Hasil OCR"]
y = df["Kategori"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)


In [66]:
from sklearn.pipeline import Pipeline

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression

# Transformer untuk bersihin kolom teks (inputnya 2D: DataFrame 1 kolom)
# 2) Define vectorizers
word_tfidf = TfidfVectorizer(
    ngram_range=(1, 2),
    max_features=10000,
    min_df=2,
    sublinear_tf=True
)

char_tfidf = TfidfVectorizer(
    analyzer="char_wb",
    ngram_range=(3, 5),
    min_df=2,
    sublinear_tf=True
)

# 3) Gabung word + char features dari kolom ke-0 (single text column)
features = ColumnTransformer(
    transformers=[
        ("word", word_tfidf, 0),
        ("char", char_tfidf, 0),
    ],
    remainder="drop"
)

# 4) Final pipeline
pipeline = Pipeline([
    ("feat", features),
    ("clf", LogisticRegression(
        max_iter=2000,
        class_weight="balanced",
        solver="lbfgs"   # atau saga
    ))
])

# alternatif pake HashingVectorizer + SGDClassifier
# from sklearn.feature_extraction.text import HashingVectorizer
# from sklearn.linear_model import SGDClassifier
# from sklearn.pipeline import FeatureUnion
# pipeline = Pipeline([
#     ("feat", FeatureUnion([
#         ("word", HashingVectorizer(
#             n_features=2**18,
#             alternate_sign=False,
#             analyzer="word",
#             ngram_range=(1,2),
#             norm="l2",
#             lowercase=True
#         )),
#         ("char", HashingVectorizer(
#             n_features=2**18,
#             alternate_sign=False,
#             analyzer="char_wb",
#             ngram_range=(3,5),
#             norm="l2",
#             lowercase=True
#         ))
#     ])),
#     ("clf", SGDClassifier(
#         loss="log_loss",
#         class_weight="balanced",
#         alpha=1e-5,
#         max_iter=3000,
#         tol=1e-3,
#         random_state=42
#     ))
# ])


In [67]:
pipeline.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('feat', ...), ('clf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('word', ...), ('char', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains s

In [68]:
from sklearn.metrics import classification_report

y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


                        precision    recall  f1-score   support

                 Bills       1.00      0.92      0.96        39
             Donations       0.92      0.87      0.89        38
             Education       1.00      0.87      0.93        38
         Entertainment       0.97      0.87      0.92        39
    Food and Beverages       0.72      0.80      0.76        45
                Health       0.92      0.95      0.93        37
                 Other       0.62      0.88      0.72        24
Savings and Investment       1.00      1.00      1.00        38
              Shopping       0.83      0.80      0.81        44
             Transport       0.79      0.79      0.79        39

              accuracy                           0.87       381
             macro avg       0.88      0.87      0.87       381
          weighted avg       0.88      0.87      0.87       381



In [69]:
import os
import joblib
import json
import sklearn

# buat folder models jika belum ada
os.makedirs("models", exist_ok=True)

# simpan seluruh pipeline
joblib.dump(pipeline, "models/tfidf_pipeline.joblib")

# simpan metadata (mis. class names, sklearn version)
meta = {
    "classifier_classes": pipeline.named_steps["clf"].classes_.tolist(),
    "sklearn_version": sklearn.__version__,
}
with open("models/tfidf_pipeline_meta.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print("Saved pipeline -> models/tfidf_pipeline.joblib")
print("Saved metadata -> models/tfidf_pipeline_meta.json")

# contoh cara load kembali (uncomment saat diperlukan):
# loaded = joblib.load("models/tfidf_pipeline.joblib")
# y_pred = loaded.predict(X_test)
# from sklearn.metrics import classification_report
# print(classification_report(y_test, y_pred))

Saved pipeline -> models/tfidf_pipeline.joblib
Saved metadata -> models/tfidf_pipeline_meta.json


In [70]:
import sklearn, inspect
from sklearn.linear_model import LogisticRegression

print("sklearn version:", sklearn.__version__)
print("LogisticRegression object:", LogisticRegression)
print("signature:", inspect.signature(LogisticRegression))


sklearn version: 1.8.0
LogisticRegression object: <class 'sklearn.linear_model._logistic.LogisticRegression'>
signature: (penalty='deprecated', *, C=1.0, l1_ratio=0.0, dual=False, tol=0.0001, fit_intercept=True, intercept_scaling=1, class_weight=None, random_state=None, solver='lbfgs', max_iter=100, verbose=0, warm_start=False, n_jobs=None)


In [71]:
import numpy as np

def get_feature_names_from_pipeline(pipeline):
    feat = pipeline.named_steps["feat"]  # ColumnTransformer
    # ambil feature names dari masing-masing transformer
    word_names = feat.named_transformers_["word"].get_feature_names_out()
    char_names = feat.named_transformers_["char"].get_feature_names_out()

    # ColumnTransformer output = [word_features | char_features] (concat)
    feature_names = np.concatenate([word_names, char_names])
    return feature_names, len(word_names), len(char_names)

def top_features_per_class(pipeline, top_k=20, only="word"):
    clf = pipeline.named_steps["clf"]
    feature_names, n_word, n_char = get_feature_names_from_pipeline(pipeline)

    classes = clf.classes_
    coefs = clf.coef_  # shape: (n_classes, n_features)

    # pilih subset fitur
    if only == "word":
        idx = np.arange(0, n_word)
    elif only == "char":
        idx = np.arange(n_word, n_word + n_char)
    else:  # "all"
        idx = np.arange(0, n_word + n_char)

    results = {}
    for i, c in enumerate(classes):
        coef = coefs[i, idx]
        names = feature_names[idx]

        top_pos = np.argsort(coef)[-top_k:][::-1]      # paling mendorong kelas c
        top_neg = np.argsort(coef)[:top_k]             # paling menentang kelas c

        results[c] = {
            "top_positive": list(zip(names[top_pos], coef[top_pos])),
            "top_negative": list(zip(names[top_neg], coef[top_neg])),
        }
    return results

res = top_features_per_class(pipeline, top_k=15, only="word")

for cls, info in res.items():
    print("\n=== Kelas:", cls, "===")
    print("Top kata yang MENDORONG kelas ini:")
    for w, v in info["top_positive"]:
        print(f"  {w:25s} {v:+.4f}")



=== Kelas: Bills ===
Top kata yang MENDORONG kelas ini:
  berjalan                  +1.2527
  periode berjalan          +1.2527
  periode                   +1.2527
  tagihan                   +1.1789
  listrik                   +0.7797
  tagihan periode           +0.7690
  rp                        +0.7111
  pembayaran tagihan        +0.6878
  idpel                     +0.6612
  total rp                  +0.6541
  rp 15                     +0.6355
  utilitas                  +0.6124
  15 000                    +0.5973
  111 000                   +0.5818
  111                       +0.5460

=== Kelas: Donations ===
Top kata yang MENDORONG kelas ini:
  donasi                    +1.3067
  lembaga                   +0.9903
  bantuan                   +0.8173
  untuk                     +0.7336
  sosial                    +0.6598
  bencana                   +0.6083
  sedekah                   +0.6023
  kemanusiaan               +0.5653
  amal                      +0.5433
  baznas          

In [72]:
# Install wandb jika belum ada
%pip install -q wandb

Note: you may need to restart the kernel to use updated packages.


In [73]:
import wandb

# ── Kumpulkan semua hyperparameter pipeline ─────────────────────────────────
clf_params  = pipeline.named_steps["clf"].get_params()
word_params = pipeline.named_steps["feat"].named_transformers_["word"].get_params()
char_params = pipeline.named_steps["feat"].named_transformers_["char"].get_params()

run_config = {
    # dataset
    "dataset_total_rows"   : len(df),
    "n_classes"            : df["Kategori"].nunique(),
    "train_size"           : len(X_train),
    "test_size"            : len(X_test),
    "test_ratio"           : 0.3,
    "random_state"         : 42,
    # word TF-IDF
    "word_ngram_range"     : str(word_params["ngram_range"]),
    "word_max_features"    : word_params["max_features"],
    "word_min_df"          : word_params["min_df"],
    "word_sublinear_tf"    : word_params["sublinear_tf"],
    # char TF-IDF
    "char_ngram_range"     : str(char_params["ngram_range"]),
    "char_min_df"          : char_params["min_df"],
    "char_sublinear_tf"    : char_params["sublinear_tf"],
    "char_analyzer"        : char_params["analyzer"],
    # classifier
    "clf_solver"           : clf_params["solver"],
    "clf_max_iter"         : clf_params["max_iter"],
    "clf_class_weight"     : str(clf_params["class_weight"]),
    "clf_C"                : clf_params["C"],
}

run = wandb.init(
    project = "monkeylimits-category-classifier",
    name    = "tfidf-logreg",
    tags    = ["tfidf", "logistic-regression", "ocr"],
    config  = run_config,
)

print("W&B run URL:", run.url)

W&B run URL: https://wandb.ai/kittyofheaven/monkeylimits-category-classifier/runs/992ud5f9


In [74]:
# ── Log distribusi kelas ─────────────────────────────────────────────────────
class_dist = df["Kategori"].value_counts()

# Akumulasi semua metrics ke satu dict — akan di-log sekaligus nanti
all_metrics = {}

# Setiap kelas sebagai scalar → langsung jadi bar chart di W&B
for cls, count in class_dist.items():
    all_metrics[f"dataset/class_count/{cls}"] = int(count)

# Summary info dataset
all_metrics["dataset/total_samples"]    = int(len(df))
all_metrics["dataset/n_classes"]        = int(df["Kategori"].nunique())
all_metrics["dataset/min_class_count"]  = int(class_dist.min())
all_metrics["dataset/max_class_count"]  = int(class_dist.max())

# Tabel distribusi (untuk export CSV)
class_dist_df = class_dist.reset_index()
class_dist_df.columns = ["Kategori", "Count"]
all_metrics["dataset/class_distribution"] = wandb.Table(dataframe=class_dist_df)

print(f"\u2705 Dataset stats siap  ({len(class_dist)} kelas)")

✅ Dataset stats siap  (10 kelas)


In [75]:
import numpy as np
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    f1_score,
)

# ── Prediksi & hitung metrik ─────────────────────────────────────────────────
y_pred      = pipeline.predict(X_test)
accuracy    = accuracy_score(y_test, y_pred)
macro_f1    = f1_score(y_test, y_pred, average="macro")
weighted_f1 = f1_score(y_test, y_pred, average="weighted")
report_dict = classification_report(y_test, y_pred, output_dict=True)
classes     = pipeline.named_steps["clf"].classes_

# Metrik utama
all_metrics["eval/accuracy"]    = accuracy
all_metrics["eval/macro_f1"]    = macro_f1
all_metrics["eval/weighted_f1"] = weighted_f1

# Per-class metrics sebagai tabel
per_class_rows = []
for cls in classes:
    m = report_dict[cls]
    per_class_rows.append([
        cls,
        round(m["precision"], 4),
        round(m["recall"], 4),
        round(m["f1-score"], 4),
        int(m["support"]),
    ])
all_metrics["eval/per_class_metrics"] = wandb.Table(
    columns=["class", "precision", "recall", "f1", "support"],
    data=per_class_rows,
)

# Per-class F1 juga sebagai scalar individual → mudah dibandingkan antar run
for cls in classes:
    all_metrics[f"eval/f1/{cls}"] = round(report_dict[cls]["f1-score"], 4)

# Confusion matrix sebagai tabel
cm = confusion_matrix(y_test, y_pred, labels=classes)
cm_data = [
    [true_cls, pred_cls, int(cm[i, j])]
    for i, true_cls in enumerate(classes)
    for j, pred_cls in enumerate(classes)
]
all_metrics["eval/confusion_matrix"] = wandb.Table(
    columns=["Actual", "Predicted", "Count"],
    data=cm_data,
)

print(f"\u2705 Eval metrics siap  |  Accuracy: {accuracy:.4f}  |  Macro F1: {macro_f1:.4f}")

✅ Eval metrics siap  |  Accuracy: 0.8714  |  Macro F1: 0.8725


In [76]:
import time
import numpy as np

# ── Ukur latency inferensi per-sample ───────────────────────────────────────
latencies_ms = []
for i in range(len(X_test)):
    sample = X_test.iloc[[i]]
    t0 = time.perf_counter()
    pipeline.predict(sample)
    t1 = time.perf_counter()
    latencies_ms.append((t1 - t0) * 1000)

latencies_ms = np.array(latencies_ms)

p50 = float(np.percentile(latencies_ms, 50))
p90 = float(np.percentile(latencies_ms, 90))
p95 = float(np.percentile(latencies_ms, 95))
p99 = float(np.percentile(latencies_ms, 99))
mean_lat = float(latencies_ms.mean())

all_metrics["latency_ms/p50"]       = p50
all_metrics["latency_ms/p90"]       = p90
all_metrics["latency_ms/p95"]       = p95
all_metrics["latency_ms/p99"]       = p99
all_metrics["latency_ms/mean"]      = mean_lat
all_metrics["latency_ms/min"]       = float(latencies_ms.min())
all_metrics["latency_ms/max"]       = float(latencies_ms.max())
all_metrics["latency_ms/histogram"] = wandb.Histogram(latencies_ms)
all_metrics["latency_ms/summary_table"] = wandb.Table(
    columns=["metric", "value_ms"],
    data=[
        ["P50 (median)", round(p50, 4)],
        ["P90",          round(p90, 4)],
        ["P95",          round(p95, 4)],
        ["P99",          round(p99, 4)],
        ["Mean",         round(mean_lat, 4)],
        ["Min",          round(float(latencies_ms.min()), 4)],
        ["Max",          round(float(latencies_ms.max()), 4)],
    ]
)

print(f"\u2705 Latency siap  |  P50: {p50:.2f} ms  |  P90: {p90:.2f} ms  |  P95: {p95:.2f} ms")

✅ Latency siap  |  P50: 1.18 ms  |  P90: 1.81 ms  |  P95: 2.97 ms


In [77]:
# ── LOG SEMUA METRICS DALAM SATU CALL ────────────────────────────────────────
# Satu wandb.log() = satu step → tidak ada masalah 'runs.history terpotong'
wandb.log(all_metrics)

print(f"\u2705 Logged {len(all_metrics)} metrics ke W&B dalam satu call.")

✅ Logged 39 metrics ke W&B dalam satu call.


In [78]:
import os

# ── Upload model sebagai W&B Artifact ────────────────────────────────────────
model_artifact = wandb.Artifact(
    name        = "tfidf-logreg-pipeline",
    type        = "model",
    description = (
        "TF-IDF (word + char n-gram) + LogisticRegression pipeline "
        "untuk klasifikasi kategori transaksi OCR"
    ),
    metadata = {
        "accuracy"    : round(accuracy, 4),
        "macro_f1"    : round(macro_f1, 4),
        "weighted_f1" : round(weighted_f1, 4),
        "n_classes"   : len(classes),
        "classes"     : list(classes),
        "p50_ms"      : round(p50, 4),
        "p95_ms"      : round(p95, 4),
    },
)

model_artifact.add_file("models/tfidf_pipeline.joblib")
model_artifact.add_file("models/tfidf_pipeline_meta.json")

run.log_artifact(model_artifact)
print("\u2705 Model artifact uploaded to W&B.")

✅ Model artifact uploaded to W&B.


In [79]:
# ── Selesaikan run ───────────────────────────────────────────────────────────
wandb.finish()
print("\U0001f389 W&B run selesai! Cek dashboard di:", run.url)

dataset/class_count/Bills,▁
dataset/class_count/Donations,▁
dataset/class_count/Education,▁
dataset/class_count/Entertainment,▁
dataset/class_count/Food and Beverages,▁
dataset/class_count/Health,▁
dataset/class_count/Other,▁
dataset/class_count/Savings and Investment,▁
dataset/class_count/Shopping,▁
dataset/class_count/Transport,▁
+24,...


🎉 W&B run selesai! Cek dashboard di: https://wandb.ai/kittyofheaven/monkeylimits-category-classifier/runs/992ud5f9
